In [4]:
from pathlib import Path
import pandas as pd
import json

In [5]:
# pqfile = "s3://avilabs-mldata-us-west-2/temp/small_train_0.parquet"
pqfile = "./small_train_0.parquet"

## Using Pandas

In [6]:
df = pd.read_parquet(pqfile, engine='pyarrow')

In [7]:
df.head()

,label,i1,i2,i3,i4,i5,i6,i7,i8,i9,...,s17,s18,s19,s20,s21,s22,s23,s24,s25,s26
0,0,1.0,1,5.0,0.0,1382.0,4.0,15.0,2,181.0,...,e5ba7672,f54016b9,21ddcdc9,b1252a9d,07b5194c,,3a171ecb,c5c50484,e8b83407,9727dd16
1,0,2.0,0,44.0,1.0,102.0,8.0,2.0,2,4.0,...,07c540c4,b04e4670,21ddcdc9,5840adea,60f6221e,,3a171ecb,43f13e8b,e8b83407,731c3655
2,0,2.0,0,1.0,14.0,767.0,89.0,4.0,2,245.0,...,8efede7f,3412118d,,,e587c466,ad3062eb,3a171ecb,3b183c5c,,
3,0,NaN,893,NaN,NaN,4392.0,NaN,0.0,0,0.0,...,1e88c74f,74ef3502,,,6b3a5ca6,,3a171ecb,9117a34a,,
4,0,3.0,-1,NaN,0.0,2.0,0.0,3.0,0,0.0,...,1e88c74f,26b3c7a7,,,21c9516a,,32c7478e,b34f3128,,


In [9]:
df["s1"].nunique()

53

## Using Spark

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import collect_list

In [2]:
spark = SparkSession.builder.getOrCreate()
spark.conf.set("spark.sql.repl.eagerEval.enabled", True)

In [6]:
df = spark.read.parquet(pqfile)

In [13]:
tp = df.groupby("s1").count()

In [14]:
type(tp)

pyspark.sql.dataframe.DataFrame

In [19]:
s2_toks = df.groupby("s2").count().sort("count", ascending=False).collect()

In [21]:
s1_toks = df.groupby("s1").count().sort("count", ascending=False).collect()

In [25]:
from collections import defaultdict

In [27]:
all_toks = {}

In [28]:
for col in ("s1", "s2"):
    all_toks[col] = {}
    toks = df.groupby(col).count().sort("count", ascending=False).collect()
    for tok, count in toks:
        all_toks[col][tok] = count

In [29]:
all_toks

{'s1': {'05db9164': 484,
  '68fd1e64': 184,
  '5a9ed9b0': 91,
  '8cf07265': 47,
  'be589b51': 39,
  '5bfa8ab5': 19,
  '87552397': 16,
  'ae82ea21': 13,
  'f473b8dc': 11,
  '39af2607': 9,
  '9a89b36c': 9,
  '09ca0b81': 7,
  '241546e0': 6,
  'b455c6d7': 5,
  '17f69355': 5,
  '3c9d8785': 4,
  '41edac3d': 3,
  '75ac2fe6': 3,
  '1a5f926e': 3,
  '287e684f': 3,
  'fb174e6b': 2,
  'ff004ae3': 2,
  '1464facd': 2,
  '439a44a4': 2,
  '7e5c2ff4': 2,
  '6ca3af46': 2,
  'fbc55dae': 1,
  '5ebc3192': 1,
  '80e4d755': 1,
  '8a033483': 1,
  '9ac48ebf': 1,
  '561bf9d4': 1,
  '9684fd4d': 1,
  '24eda356': 1,
  '43b5ce4b': 1,
  '3b65d647': 1,
  '65aada8c': 1,
  'cf1f182b': 1,
  'd4b08d58': 1,
  'd9baf3a8': 1,
  'dbe63c2b': 1,
  '6e4a368f': 1,
  '9660b97b': 1,
  '45cb84c9': 1,
  '68a25dc5': 1,
  '4a4e85c4': 1,
  '64e77ae7': 1,
  '36a5b3ff': 1,
  '8c6ba407': 1,
  'f0a33555': 1,
  'c512b859': 1,
  'a86f8721': 1,
  'f434fac1': 1},
 's2': {'38a947a1': 109,
  '09e68b86': 72,
  '80e26c9b': 51,
  'd833535f': 43,
  

In [32]:
df.groupby("s1", "s2").count()

s1,s2,count
05db9164,287130e0,17
68fd1e64,dd8c896e,1
41edac3d,09e68b86,1
09ca0b81,f0cf0024,1
05db9164,fe5f56f0,1
68fd1e64,6582398b,1
05db9164,dd8c896e,1
1a5f926e,80e26c9b,1
39af2607,0a519c5c,1
05db9164,9bcd4a15,1
